# Tic Tac Toe Coin On Blockchain Simulator

<img src="creating-coin.jpg" alt="Create Tic Tac Toe Coin" width="600"/>

In [1]:
%%bash
chia version
cdv --version
python --version

1.5.1.dev0
cdv, version 1.0.8
Python 3.10.5


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G2Element)

from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.coin import Coin
from chia.types.blockchain_format.program import Program
from chia.types.coin_spend import CoinSpend
from chia.types.condition_opcodes import ConditionOpcode
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle

from clvm.casts import int_to_bytes
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle, print_json, print_push_tx_result)

sys.path.insert(0, "./code")
import tic_tac_toe

# load puzzles
tic_tac_toe_puzzle = load_program("./code/tic-tac-toe.clsp", ["./code", "../../../shared"])
coin_puzzle = load_program("./code/coin.clsp", ["./code", "../../../shared"])
terminate_puzzle = load_program("./code/terminate-game.clsp", ["./code", "../../../shared"])

## 1. Set Up A Blockchain Simulator

In [11]:
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_records_by_puzzle_hash = network.sim_client.get_coin_records_by_puzzle_hash
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
bob: Wallet = network.make_wallet("bob")
await network.farm_block(farmer=alice)
await network.farm_block(farmer=bob)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice pk:\t\t{alice.pk()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')
print(f'bob balance:\t\t{bob.balance()}')
print(f'alice pk:\t\t{alice.pk()}')
print(f'bob puzzle hash:\t{bob.puzzle_hash}')

# prepare players info
player_one_info = Program.to([alice.pk(), alice.puzzle_hash])
player_two_info = Program.to([bob.pk(), bob.puzzle_hash])

alice balance:		2000000000000
alice pk:		aba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3
bob balance:		2000000000000
alice pk:		aba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8
bob puzzle hash:	87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5


## 2. Alice & Bob Create Tic Tac Toe Coin Together

In [12]:
# both players put one XCH
player_amount = 1_000_000_000_000
player_fee = 50_000_000
p2_amount = player_amount - player_fee
game_amount = player_amount * 2

# prepare terminate-game puzzle
# (mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
curried_terminate_puzzle = terminate_puzzle.curry(
    0,
    alice.puzzle_hash,
    bob.puzzle_hash,
    p2_amount
)

def get_coin_puzzle(board, player):
    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        curried_terminate_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        game_amount)
    return curried_coin_puzzle

curried_coin_puzzle = get_coin_puzzle([' '] * 9, 'x')
curried_coin_puzzle_hash = curried_coin_puzzle.get_tree_hash()

In [13]:
alice_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
alice_coin = alice_coin_wrapper.as_coin()
alice_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())
# alice's coin create tic tac toe coin
alice_conditions = [
    [ConditionOpcode.CREATE_COIN, curried_coin_puzzle_hash, game_amount],
    [ConditionOpcode.CREATE_COIN, alice.puzzle_hash, alice_coin.amount - player_amount],
]

alice_delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_conditions(alice_conditions)
alice_delegated_puzzle_solution = p2_delegated_puzzle_or_hidden_puzzle.solution_for_conditions(alice_conditions)
alice_coin_spend = CoinSpend(
    alice_coin,
    alice_puzzle,
    alice_delegated_puzzle_solution
)

alice_coin_message = (
    alice_delegated_puzzle.get_tree_hash()
    + alice_coin.name()
    + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
)
alice_synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)
alice_signature: G2Element = AugSchemeMPL.sign(
    alice_synthetic_sk,
    alice_coin_message
)

In [14]:
bob_coin_wrapper = await bob.choose_coin(1_750_000_000_000)
bob_coin = bob_coin_wrapper.as_coin()
bob_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(bob.pk())
# bob's coin create change back to himself
bob_conditions = [
    [ConditionOpcode.CREATE_COIN, bob.puzzle_hash, bob_coin.amount - player_amount],
]

bob_delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_conditions(bob_conditions)
bob_delegated_puzzle_solution = p2_delegated_puzzle_or_hidden_puzzle.solution_for_conditions(bob_conditions)
bob_coin_spend = CoinSpend(
    bob_coin,
    bob_puzzle,
    bob_delegated_puzzle_solution
)

bob_coin_message = (
    bob_delegated_puzzle.get_tree_hash()
    + bob_coin.name()
    + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
)
bob_synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    bob.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)
bob_signature: G2Element = AugSchemeMPL.sign(
    bob_synthetic_sk,
    bob_coin_message
)

In [15]:
agg_sig = AugSchemeMPL.aggregate([alice_signature, bob_signature])
spend_bundle = SpendBundle([alice_coin_spend, bob_coin_spend], agg_sig)
#print_json(spend_bundle.to_json_dict())
await network.push_tx(spend_bundle)

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 8637d94359c5b33e714d07e707a1bf8772e4e1ee432439476359f6b6aaa65561>, amount=2000000000000),
  Coin(parent_coin_info=<bytes32: 17ca02c0a209d7e1a3869442ba13ef9468181c4b095b8823aeaf3c27f8e58c34>, puzzle_hash=<bytes32: 87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5>, amount=750000000000)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000002>, puzzle_hash=<bytes32: 8

In [16]:
network.sim.pass_blocks(1)
print(f'alice balance:\t\t{alice.balance()}')
print(f'bob balance:\t\t{bob.balance()}')

coin_records = await get_coin_records_by_puzzle_hash(curried_coin_puzzle_hash)
tic_tac_toe_coin = coin_records[0].coin
print(tic_tac_toe_coin)

alice balance:		1000000000000
bob balance:		1000000000000
{'amount': 2000000000000,
 'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
 'puzzle_hash': '0x8637d94359c5b33e714d07e707a1bf8772e4e1ee432439476359f6b6aaa65561'}


## 3. Simulate Players Taking Turns

In [17]:
async def sim_play(tic_tac_toe_coin, sk, board, player, position):
    curried_coin_puzzle = get_coin_puzzle(board, player)
    curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_puzzle_from_curried_coin_puzzle(curried_coin_puzzle)
    board_state, new_board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
    tic_tac_toe.print_board(new_board)

    coin_message = (
        std_hash(int_to_bytes(position))
        + tic_tac_toe_coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )

    # sign with sk
    signature: G2Element = AugSchemeMPL.sign(
        sk,
        coin_message
    )

    coin_spend = CoinSpend(
        tic_tac_toe_coin,
        curried_coin_puzzle,
        Program.to([position]) # position
    )

    spend_bundle = SpendBundle([coin_spend], signature)
    #print_json(spend_bundle.to_json_dict())
    result = await network.push_tx(spend_bundle)
    print_push_tx_result(result)
    tic_tac_toe_coin = result["additions"][0]
    #print(tic_tac_toe_coin)
    next_player = 'x' if player == 'o' else 'o'
    return new_board, next_player, tic_tac_toe_coin

## 4. Simulate Winning Board
1. Start with empty board.
2. Alice (`x`) plays first.
3. Bob (`o`) and Alice (`x`) take turn until Alice wins.

In [10]:
board = [' '] * 9
player = 'x'
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, alice.sk_, board, player, 4)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, bob.sk_, board, player, 0)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, alice.sk_, board, player, 3)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, bob.sk_, board, player, 1)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, alice.sk_, board, player, 5)

   |   |   
---+---+---
   | x |   
---+---+---
   |   |   

additions:
{'amount': 2000000000000,
 'parent_coin_info': '0x37b4ec70323155e7d587a84d880b0cc68c3c85c6d106808e89278dac165a6974',
 'puzzle_hash': '0x823e26d76fd3ca6607fac6e1212b850a0e68f9755d288d02dd16a81a36256a4b'}
removals:
{'amount': 2000000000000,
 'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
 'puzzle_hash': '0x8637d94359c5b33e714d07e707a1bf8772e4e1ee432439476359f6b6aaa65561'}
 o |   |   
---+---+---
   | x |   
---+---+---
   |   |   

additions:
{'amount': 2000000000000,
 'parent_coin_info': '0xc73dffc6dad7247b3a5db533737d85e96e63947f743ee04b77dac2903e43c2a6',
 'puzzle_hash': '0x9dc6ea6cd33f033d7c13443e62b5e25d324bfca6cee1ef5881e11a750a04d97b'}
removals:
{'amount': 2000000000000,
 'parent_coin_info': '0x37b4ec70323155e7d587a84d880b0cc68c3c85c6d106808e89278dac165a6974',
 'puzzle_hash': '0x823e26d76fd3ca6607fac6e1212b850a0e68f9755d288d02dd16a81a36256a4b'}
 o |   |   
---+---+---


## 5. Simulate Tie Game
> re-run step 1-3 again
1. Start with empty board.
2. `x` plays first.
3. `o` and `x` take turn until the board is not playable.

In [18]:
board = [' '] * 9
player = 'x'
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, alice.sk_, board, player, 0)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, bob.sk_, board, player, 1)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, alice.sk_, board, player, 2)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, bob.sk_, board, player, 4)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, alice.sk_, board, player, 5)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, bob.sk_, board, player, 6)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, alice.sk_, board, player, 7)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, bob.sk_, board, player, 8)
board, player, tic_tac_toe_coin = await sim_play(tic_tac_toe_coin, alice.sk_, board, player, 3)


 x |   |   
---+---+---
   |   |   
---+---+---
   |   |   

additions:
{'amount': 2000000000000,
 'parent_coin_info': '0x37b4ec70323155e7d587a84d880b0cc68c3c85c6d106808e89278dac165a6974',
 'puzzle_hash': '0x500ec0712274f55b62e2ebfbacad6816bf98030f82f69e8d34a98aca64a0c1f4'}
removals:
{'amount': 2000000000000,
 'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
 'puzzle_hash': '0x8637d94359c5b33e714d07e707a1bf8772e4e1ee432439476359f6b6aaa65561'}
 x | o |   
---+---+---
   |   |   
---+---+---
   |   |   

additions:
{'amount': 2000000000000,
 'parent_coin_info': '0x793098a854caafa3b6c1960cae9ff24bd6462fbabc1ca67684e26f447156e598',
 'puzzle_hash': '0x7bc24e8483ecb5029ec85acf8829eb506a469c5089139148fe4fc7a8e7d76dfa'}
removals:
{'amount': 2000000000000,
 'parent_coin_info': '0x37b4ec70323155e7d587a84d880b0cc68c3c85c6d106808e89278dac165a6974',
 'puzzle_hash': '0x500ec0712274f55b62e2ebfbacad6816bf98030f82f69e8d34a98aca64a0c1f4'}
 x | o | x 
---+---+---
